In [10]:
pip install --upgrade pydot pyparsing

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
import pydot
import random
import math
import matplotlib.pyplot as plt
import networkx as nx
import math
from statistics import mean 
from collections import Counter
import numpy as np
from numpy import linalg as LA
import copy

In [45]:
def parse_dot_file(file_path):
    graph = pydot.graph_from_dot_file(file_path)[0]  # Get the first graph from the list
    nodes = []
    edges = []
    for node in graph.get_nodes():
        nodes.append(node.get_name())
    for edge in graph.get_edges():
        source = edge.get_source()
        target = edge.get_destination()
        attributes = edge.get_attributes()
        edges.append((source, target, attributes))
    return nodes, edges

def parse_subgraph_dot_file(file_path):
    # a string version so you can play around with it easier
    graph_string = pydot.graph_from_dot_file(file_path)[0].to_string()
    graph = nx.nx_pydot.read_dot(file_path)
    # fastest way to get nodes is with networkx, pydot does not understand subgraphs
    nodes = graph.nodes()
    edges = graph.edges()
    
    # Remove 'n' prefix from nodes
    nodes = [node[1:] for node in nodes]

    # Remove 'n' prefix from edges
    edges = [(u[1:], v[1:]) for u, v in edges]
    return nodes, edges, graph_string

file_path = r"D:/OneDrive - Universiteit Utrecht/Universiteit/3DataVisualisation/DataSet/devonshiredebate.dot"

#nodes, edges = parse_dot_file(file_path)
nodes, edges, G = parse_subgraph_dot_file(file_path)

[('10', '11'), ('11', '171'), ('11', '173'), ('11', '174'), ('14', '15'), ('14', '230'), ('15', '37'), ('18', '19'), ('19', '166'), ('20', '21'), ('23', '21'), ('27', '24'), ('27', '17'), ('27', '28'), ('27', '29'), ('24', '207'), ('17', '224'), ('28', '26'), ('28', '131'), ('28', '124'), ('28', '132'), ('28', '242'), ('28', '246'), ('33', '34'), ('33', '55'), ('33', '167'), ('33', '168'), ('33', '172'), ('33', '173'), ('34', '56'), ('34', '159'), ('34', '177'), ('34', '178'), ('35', '13'), ('35', '11'), ('35', '57'), ('35', '157'), ('35', '160'), ('38', '37'), ('38', '15'), ('38', '155'), ('39', '15'), ('39', '3'), ('39', '204'), ('40', '15'), ('40', '175'), ('40', '231'), ('40', '232'), ('43', '42'), ('43', '44'), ('43', '158'), ('45', '44'), ('45', '58'), ('45', '60'), ('45', '177'), ('44', '42'), ('46', '44'), ('46', '45'), ('46', '61'), ('46', '168'), ('48', '47'), ('48', '19'), ('48', '163'), ('51', '50'), ('51', '52'), ('51', '161'), ('53', '52'), ('53', '179'), ('52', '50'), ('

### Step 1

In [13]:
# Function to arrange nodes in a square-like layout
def arrange_in_square(nodes):
    num_nodes = len(nodes)
    side_length = int(math.ceil(math.sqrt(num_nodes)))
    spacing = 50
    start_x, start_y = -((side_length - 1) * spacing) / 2, ((side_length - 1) * spacing) / 2
    node_positions = {}
    current_x, current_y = start_x, start_y
    for i, node in enumerate(nodes):
        node_positions[node] = (current_x, current_y)
        current_x += spacing

        if (i + 1) % side_length == 0:
            current_x = start_x
            current_y -= spacing  # Move to the next row
    return node_positions

def arrange_in_radial(nodes):
    num_nodes = len(nodes)
    radius = 200
    angle_step = 360 / num_nodes
    node_positions = {}
    for i, node in enumerate(nodes):
        angle = math.radians(i * angle_step)
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        node_positions[node] = (x, y)
    return node_positions

def arrange_in_semi_square(nodes):
    num_nodes = len(nodes)
    side_length = int(math.ceil(math.sqrt(num_nodes)))  # Calculate the side length of the square
    spacing = 60  # Spacing between nodes
    start_x, start_y = -((side_length - 1) * spacing) / 2, ((side_length - 1) * spacing) / 2
    node_positions = {}
    current_x, current_y = start_x, start_y
    for i, node in enumerate(nodes):
        node_positions[node] = (current_x, current_y + random.randint(-15, 15))
        current_x += spacing

        if (i + 1) % side_length == 0:
            current_x = start_x
            current_y -= spacing  # Move to the next row

    return node_positions

In [14]:
def draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size=300, directed=False): 
    fig, ax = plt.subplots()
               
    # Draw nodes and labels
    for node, pos in node_positions.items():
        if 'd' in node: #if it is a dummy
            ax.scatter(pos[0], pos[1], s=node_size/2, color=node_colors[node], edgecolors='black', zorder=5, marker="s")  # Draw node
        elif not '|' in node:
            ax.scatter(pos[0], pos[1], s=node_size, color=node_colors[node], edgecolors='black', zorder=5)  # Draw node
        ax.text(pos[0], pos[1], node[0:4], ha='center', va='center', fontsize=8, zorder=10)  # Add label
        
    # Draw edges
    for edge in edges:
        source, target, _ = edge
        start = node_positions[source]
        end = node_positions[target]
        if 'd' in source: #if it is a dummy
            original_source = source
            while 'd' in original_source:
                original_source = [other_source for other_source, other_target, weight in edges if other_target == source][0]
                source = original_source
            ax.plot([start[0], end[0]], [start[1], end[1]], color=node_colors[original_source], zorder=0)  # Draw edge
        elif directed: #TODO YVETTE
             ax.arrow(start[0], start[1], end[0]-start[0], end[1]-start[1], length_includes_head=True, head_width = np.sum(np.abs(ax.get_xlim()))/50, color=node_colors[source], zorder=0) # Draw arrow
        else:
            ax.plot([start[0], end[0]], [start[1], end[1]], color=node_colors[source], zorder=0)  # Draw edge
            
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)    
    plt.show()

In [53]:
# FOR SUBGRAPHS
def draw_two_node_link_diagram(edges1, edges2, node_positions1, node_positions2, node_colors, node_size=300, directed=False): 
    fig, (ax1,ax2) = plt.subplots(1,2)
               
    # Draw nodes and labels
    for node, pos in node_positions1.items():
        if 'd' in node: #if it is a dummy
            ax1.scatter(pos[0], pos[1], s=node_size/2, color=node_colors[node], edgecolors='black', zorder=5, marker="s")  # Draw node
        elif not '|' in node:
            ax1.scatter(pos[0], pos[1], s=node_size, color=node_colors[node], edgecolors='black', zorder=5)  # Draw node
        ax1.text(pos[0], pos[1], node[0:4], ha='center', va='center', fontsize=8, zorder=10)  # Add label
    
    for node, pos in node_positions2.items():
        if 'd' in node: #if it is a dummy
            ax2.scatter(pos[0], pos[1], s=node_size/2, color=node_colors[node], edgecolors='black', zorder=5, marker="s")
        elif not '|' in node:
            ax2.scatter(pos[0], pos[1], s=node_size, color=node_colors[node], edgecolors='black', zorder=5)
        ax2.text(pos[0], pos[1], node[0:4], ha='center', va='center', fontsize=8, zorder=10)    
    
    # Draw edges
    for edge in edges1:
        source, target = edge
        start = node_positions1[source]
        end = node_positions1[target]
        if 'd' in source: #if it is a dummy
            original_source = source
            while 'd' in original_source:
                original_source = [other_source for other_source, other_target, weight in edges if other_target == source][0]
                source = original_source
            ax1.plot([start[0], end[0]], [start[1], end[1]], color=node_colors[original_source], zorder=0)  # Draw edge
        elif directed: 
             ax1.arrow(start[0], start[1], end[0]-start[0], end[1]-start[1], length_includes_head=True, head_width = np.sum(np.abs(ax.get_xlim()))/50, color=node_colors[source], zorder=0) # Draw arrow
        else:
            ax1.plot([start[0], end[0]], [start[1], end[1]], color=node_colors[source], zorder=0)  # Draw edge
            
    for edge in edges2:
        source, target = edge
        start = node_positions2[source]
        end = node_positions2[target]
        if 'd' in source: #if it is a dummy
            original_source = source
            while 'd' in original_source:
                original_source = [other_source for other_source, other_target, weight in edges if other_target == source][0]
                source = original_source
            ax2.plot([start[0], end[0]], [start[1], end[1]], color=node_colors[original_source], zorder=0)  # Draw edge
        elif directed: 
             ax2.arrow(start[0], start[1], end[0]-start[0], end[1]-start[1], length_includes_head=True, head_width = np.sum(np.abs(ax.get_xlim()))/50, color=node_colors[source], zorder=0) # Draw arrow
        else:
            ax2.plot([start[0], end[0]], [start[1], end[1]], color=node_colors[source], zorder=0)  # Draw edge
            
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)
    plt.show()

In [16]:
# Main function to draw the node-link diagram using Matplotlib
def main(nodes, edges, structure='square', node_size=300, root=None, search_algorithm=False, search_root=None, directed=False): 
    #add random color to each node (as node:colorvalue)
    node_colors = {}
    for node in nodes:
        node_colors[node] = "#" + ''.join(random.choices('0123456789ABCDEF', k=6)) #create random 6 hex code for color (which networkx uses)

    # Arrange nodes in a structure
    if structure =='square':
        positions = arrange_in_square(nodes)
    elif structure == 'radial':
        positions = arrange_in_radial(nodes)
    elif structure == 'semi_square':
        positions = arrange_in_semi_square(nodes)
    elif structure == 'tree':
        if directed: 
            positions = sugiyama_framework(nodes, edges, node_colors, root, node_size)
        else:
            positions = arrange_in_tree(nodes, edges, root)
    elif structure == 'force':
        positions = arrange_in_force(nodes,edges)
    elif structure == 'subgraph':
        positions = arrange_in_subgraphs(nodes, edges, G)
    else:
        print('Structure typed incorrectly (square, radial, semi_square or tree) OR chosen structure not undirected')
    
    if search_algorithm and structure == 'tree':
        search_through_tree(nodes, edges, positions, node_colors, search_root, node_size)
    if not directed:
        print("(Original) graph:")
        draw_node_link_diagram(nodes, edges, positions, node_colors, node_size, directed)     

### Step 2

#### Tree

In [17]:
def arrange_in_tree(nodes, edges, root=None):
    num_nodes = len(nodes)
    node_positions = {}
    if root == None:
        root = nodes[0]
        
    node_positions[root] = (0, 0) 
    layer_space = 1
    node_space = 1
    padding = 10

    queue = [root]
    past_nodes = []
    '''Tree creation'''
    while queue != []:
        current_node = queue[0]
        connected_edges = [t[:2] for t in edges if current_node in t[:2]] 
        children_nodes = [value for tuple in connected_edges for value in tuple if value != current_node and value not in past_nodes]
        if children_nodes != []: 
            segment_length = node_space / len(children_nodes)
            parent_coords = node_positions[current_node]
            for i, children_node in enumerate(children_nodes):
                if children_node not in node_positions:
                    new_x = -node_space/2+segment_length*(0.5+i)
                    new_y = parent_coords[1]-layer_space
                    node_positions[children_node] = (new_x, new_y)
            queue = children_nodes + queue
        queue.remove(current_node)
        past_nodes.append(current_node)
        
        
    '''Improve node positions in tree '''
    #this part will calculate what the max and min x&y-coordinate for the graph to determine the most rightest & leftest etc. place
    layer_min_max = {} 
    for node, node_pos in node_positions.items():
        layer = node_pos[1] #y-as
        x_coord = node_pos[0] #x-as
        if layer not in layer_min_max:
            layer_min_max[layer] = [x_coord, x_coord]
        else:
            if x_coord < layer_min_max[layer][0]:
                layer_min_max[layer][0] = x_coord
            if x_coord > layer_min_max[layer][1]:
                layer_min_max[layer][1] = x_coord

    # Move nodes which are already in the same layer and divide them equally through the layer
    for layer, (min_x, max_x) in layer_min_max.items():
        layer_nodes = [(node, pos) for node, pos in node_positions.items() if pos[1] == layer]
#         if len(layer_nodes) > 5:
        if len(layer_nodes) > 2: #TODO fix visual
            interval = (max_x - min_x) / (len(layer_nodes) - 1)
            for i, (node, pos) in enumerate(layer_nodes):
                new_x = min_x + i * interval
                node_positions[node] = (new_x, pos[1])
#             print(node_positions)
        elif layer_nodes[0][1][1] < 0: #check if not first layer and not a lot of nodes, connected them with their parents by giving the parent's node x-as
            for i, layer_node in enumerate(layer_nodes):
                parent_node = [parent_node[0] for parent_node in edges if parent_node[1] == layer_node[0]]
                middle_parent_node_coords = mean([parent_node_coords[1][0] for parent_node_coords in previous_layer_nodes if parent_node_coords[0] in parent_node])
                node_positions[layer_node[0]] = (middle_parent_node_coords, layer_node[1][1])
        layer_nodes = [(node, pos) for node, pos in node_positions.items() if pos[1] == layer]        
        previous_layer_nodes = layer_nodes.copy()
    return node_positions

#### Search Algorithms

In [18]:
def DFS(nodes, edges, search_root=None, node_size=200):
    '''DFS'''
    num_nodes = len(nodes)
    if search_root == None:
        search_root = nodes[0]
    stack = [search_root]
    visited = []
    DFS_edges = []
    while stack != []:
        current_node = stack[0]
        connected_edges = [t[:2] for t in edges if current_node in t[:2]] 
        connected_edges = sorted(connected_edges, key=lambda x: int(x[1]))
        children_nodes = [value for tuple in connected_edges for value in tuple if value != current_node and value not in visited]        
        if children_nodes != []:
            for children_node in children_nodes:
                if children_node in stack:
                    stack.remove(children_node)
                stack[stack.index(current_node):stack.index(current_node)] = [children_node]
            for edge in edges:
                if (edge[0] == current_node and edge[1] == children_nodes[0] 
                     and edge[1] not in [elem[1] for elem in DFS_edges]): #same source and if target is first edge
                    DFS_edges.append(edge)
                elif (edge[1] == current_node and edge[0] == children_nodes[0]
                      and edge[0] not in [elem[0] for elem in DFS_edges]): #viceversa cuz undirected
                    DFS_edges.append((edge[1], edge[0], edge[2]))
        elif len(visited) != len(nodes):
            stack.insert(0,[elem[0] for elem in DFS_edges if elem[1] == current_node][0])
        if current_node not in visited:
            visited.append(current_node)
        stack.remove(current_node)
    return DFS_edges, visited

def BFS(nodes, edges, search_root=None, node_size=200):
    '''BFS'''
    BFS_edges = []
    num_nodes = len(nodes)
    if search_root == None:
        search_root = nodes[0]
    queue = [search_root]
    layers = [[search_root]]
    visited = []
    while queue != []:
        current_node = queue[0]
        connected_edges = [t[:2] for t in edges if current_node in t[:2]] 
        connected_edges = sorted(connected_edges, key=lambda x: int(x[1]))
        parent_nodes = [value for edge in connected_edges for value in edge if value != current_node and value not in visited and current_node != edge[0]]
        children_nodes = [value for edge in connected_edges for value in edge if value != current_node and value not in visited and current_node != edge[1]]
        connected_nodes = parent_nodes + children_nodes
        print("layers", layers, "parent", parent_nodes, "child", children_nodes, "conn", connected_nodes)
        if connected_nodes != []:
            for connected_node in connected_nodes:
                if connected_node not in queue:
                    queue.append(connected_node)
                for edge in edges:
                    if (edge[0] == current_node and edge[1] == connected_node
                       and edge[1] not in [elem[1] for elem in BFS_edges]): #same source and if target is first edge
                        BFS_edges.append(edge)
                    elif (edge[1] == current_node and edge[0] == connected_node
                          and edge[0] not in [elem[0] for elem in BFS_edges]): #viceversa cuz undirected
                        BFS_edges.append((edge[1], edge[0], edge[2]))
        if current_node not in visited:
            visited.append(current_node)
            for i, layer in enumerate(layers):
                if current_node in layer:
                    for connected_node in connected_nodes:
                        if connected_node in parent_nodes:
                            layers[i-1].append(connected_node) #parent
                        else:
                            try:
                                layers[i+1].append(connected_node) #child
                            except IndexError:
                                layers.append([connected_node])
        queue.remove(current_node)
        if queue == [] and len(visited) < len(nodes): #if multiple trees
            not_visited = [x for x in nodes if x not in visited]
            layers.append(['|' , not_visited[0]])
            queue = [not_visited[0]] 
    return BFS_edges, visited, layers


    
def search_through_tree(nodes, edges, node_positions, node_colors, search_root=None, node_size=200):
#     print("DFS")
#     DFS_edges, DFS_visited = DFS(nodes, edges, search_root) 
#     print('DFS Visit order', DFS_visited)
#     DFS_node_positions = arrange_in_tree(nodes, DFS_edges, search_root)
#     draw_node_link_diagram(DFS_visited, DFS_edges, DFS_node_positions, node_colors, node_size)
#     print('DFS removed edges: ', [x[:2] for x in edges if x not in DFS_edges])
    
    print("BFS")
    BFS_edges, BFS_visited, layers = BFS(nodes, edges, search_root) 
    print('BFS Visit order', BFS_visited)
    BFS_node_positions = arrange_in_tree(nodes, BFS_edges, search_root)
    draw_node_link_diagram(BFS_visited, BFS_edges, BFS_node_positions, node_colors, node_size)
    print('BFS removed edges: ', [x[:2] for x in edges if x not in BFS_edges])

### Step 3

In [ ]:
def arrange_in_force(nodes, edges):
    # choice of forces/parameters
    force_magnitude = 2
    k = 3
    l = 1
    delta = 1
    cspring = 2
    crep = 1
    
    node_positions = arrange_in_semi_square(nodes)
    num_nodes = len(nodes)
    
    t = 1
    while t < k:
        check = 1
        for node in nodes:
            fspring = []
            frep = []
            current = 1
            while current <= num_nodes:
                if check != current:
                    p_u = np.array(node_positions[node])
                    p_v = np.array(node_positions[f'{nodes[current-1]}'])

                    direction = p_v - p_u
                    direction_unit = direction / np.linalg.norm(direction)
                    force = force_magnitude * direction_unit
                    p_u = p_u + force*delta

                    if (node, f'{current}', {}) in edges:
                        adjecent = True
                    else:
                        adjecent = False
                        
                    # Spring Embedder
                    if adjecent==True:
                        fspring.append(cspring / LA.norm(direction)**2 * pointing(p_v,p_u))
                    elif adjecent==False:
                        frep.append(crep * math.log(LA.norm(direction)/l) * pointing(p_v,p_u))

                current += 1
            Fv = sum(fspring) + sum(frep)
            node_positions[node] = tuple(Fv)
            check += 1
        t += 1
    return node_positions

def pointing(p1,p2):
    return p2-p1 / LA.norm(p2-p1)

In [30]:
main(nodes, edges, "force", node_size=200, directed=False)

KeyError: 1

In [21]:
def fruchterman_and_reingold(nodes,edges):
    # l = C * sqrt(area/number of vertices) - l
    l = 1
    delta = 1
    cspring = 2
    crep = 1
    # all node pairs
    frep = (l**2 / LA.norm(p2-p1)) * pointing(p1,p2)
    # adjecent nodes
    fattr = (LA.norm(p2-p1)**2 / l) * pointing(p1,p2)
    # resulting
    fspring = frep + fattr
    return fspring

### Step 4

In [22]:
def get_parents(edges, node, parents=None, parent_number=None):
    if parents is None:
        parents = {}
    if parent_number is None: #parent_number is the number of layer it is above the current node
        parent_number = 0
        
    for edge in edges:
        if edge[1] == node:
            parent = edge[0]
            if parent not in parents:
                parent_number += 1
                parents[parent] = parent_number
                get_parents(edges, parent, parents, parent_number)
    
    return parents

In [23]:
def get_childs(edges, node, childs=None, child_number=None):
    if childs is None:
        childs = {}
    if child_number is None:
        child_number = 0
        
    for edge in edges:
        if edge[0] == node:
            child = edge[1]
            if child not in childs:
                child_number -= 1
                childs[child] = child_number
                get_childs(edges, node, childs, child_number)
    
    return childs

In [24]:
def sugiyama_framework(nodes, edges, node_colors, root=None, node_size=200):
    def guarantee_heuristic(nodes, edges, node_positions):
        '''Find the edges to be reversed to avoid cyclic graph'''
        print("Heuristic with guarantees")
        #Part 1: Sinks (only receiving edges) 
        original_nodes = set(nodes) #turn the nodes to set, to use the handy set functions
        while True:
            sinks_edges = {node: [(source, target, weight) for source, target, weight in edges if target == node] for node in set([edge[1] for edge in edges]) - set([edge[0] for edge in edges])}
            removable_edges = [element for innerList in sinks_edges.values() for element in innerList]
            edges = [edge for edge in edges if edge not in removable_edges]             # Remove edges corresponding to the sink nodes
            current_nodes = set([node for edge in edges for node in edge[:2]]) #new nodes
            if current_nodes == original_nodes: # if the current set of nodes is the same as the original set, exit loop
                break
            else: # otherwise, update the set of nodes for the next iteration
                original_nodes = current_nodes
                nodes = list(original_nodes) #turn it back to the original list type
#         print("Removed sinks:")
#         draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)       
        
        
        #Part 2: Sources (only outcoming edges)
        original_nodes = set(nodes) #turn the nodes to set, to use the handy set functions
        while True:
            sources_edges = {node: [(source, target, weight) for source, target, weight in edges if source == node] for node in set([edge[0] for edge in edges]) - set([edge[1] for edge in edges])}
            removable_edges = [element for innerList in sources_edges.values() for element in innerList]
            edges = [edge for edge in edges if edge not in removable_edges]
            current_nodes = set([node for edge in edges for node in edge[:2]])
            if current_nodes == original_nodes:
                break
            else:
                original_nodes = current_nodes
                nodes = list(original_nodes)
#         print("Removed sources:")
#         draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)

        #Part 3: Inversing the edges
        # Create a Counter to count the occurrences of each node in the edge structure (+1 if it a source -1 if it a target)
        node_counter = Counter()
        for source, target, weight in edges:
            node_counter[source] += 1 # N ->
            node_counter[target] -= 1 # N <-
        nodes = sorted(node_counter, key=node_counter.get, reverse=True) #get counter in reversed aka descending order
        cycle_edges = []
        for node in nodes:
            temp_edges = []
            for edge in edges:
                if edge[0] != node or edge in cycle_edges:
                    temp_edges.append(edge)
                if edge[1] == node:
                    cycle_edges.append(edge)
            edges = temp_edges
            
#         print("Found cycle edges")
#         draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)
        return edges
            
    def layer_assignment(nodes, edges, root=None):
        #add to layers from root
        if root == None:
            root = nodes[0]
        BFS_edges, BFS_visited, layers = BFS(nodes, edges, root) 
        print('BFS Visit order', BFS_visited)
        print('BFS removed edges: ', [x[:2] for x in edges if x not in BFS_edges])
        node_positions = {}
        layer_counter = 0
        node_space = 0
        placed_nodes_in_layers = copy.deepcopy(layers)
        for index, layer in enumerate(layers):
            if "|" in layer:
                layer_counter = 0
            for node in layer: #loops through the layers
                if node not in node_positions:  #if the first time encountering a node place it
                    node_positions[node] = (node_space, layer_counter)
                    node_space += 1                  
                else: #if not, then also move every parent from that node to the current position of the node
                    node_parents = get_parents(edges, node)
                    for node_parent, parent_layer in node_parents.items():
                        if node_parent in node_positions:
                            node_positions[node_parent] = (node_positions[node_parent][0], node_positions[node][1] + parent_layer)
                        else:
                            node_positions[node_parent] = (node_space, node_positions[node][1] + parent_layer)
            layer_counter -= 1
        #Adjusting the nodes which have parents/childs that have only dummies in between them
        for node, (x, y) in node_positions.items():
            node_childs = [key for key, value in get_childs(edges, node).items() if value <= 2] 
            node_parents = [key for key, value in get_parents(edges, node).items() if value <= 2] 
            if list(node_childs) != []:
                if y-1 != max(node_positions[node_child][1] for node_child in node_childs): 
                    #if the max y-as of a child isn't directly under the current node
                    node_positions[node] = (x, y-1) #move it down
            if list(node_parents) != []:
                if y+1 != min(node_positions[node_parent][1] for node_parent in node_parents): 
                    #if the min y-as of a parent isn't directly above the current node
                    node_positions[node] = (x, y+1) #move it up
        
        #Add dummies
        layer_crossing_edges = [edge for edge in edges if abs(node_positions[edge[0]][1] - node_positions[edge[1]][1]) >= 2]
        dummy_number = 0
        while layer_crossing_edges != []:
            for index, edge in enumerate(layer_crossing_edges):
                dummy_number += 1
                dummy_name = 'd'+str(dummy_number)
                nodes.append(dummy_name) #add dummy to node list
                node_colors[dummy_name] = "#808080" #add its color 
                node_positions[dummy_name] = (((node_positions[edge[0]][0] + node_positions[edge[1]][0])/2), round((node_positions[edge[0]][1] + node_positions[edge[1]][1])/2))
                edges.append((edge[0], dummy_name, edge[2]))
                edges.append((dummy_name, edge[1], edge[2]))
                edges.remove(edge)
            layer_crossing_edges = [edge for edge in edges if abs(node_positions[edge[0]][1] - node_positions[edge[1]][1]) >= 2]

        return nodes, edges, node_positions
    
    def crossing_minimization(nodes, edges, node_positions, method="Barycenter"):
        layers = sorted(set(y for _, (_, y) in node_positions.items()), reverse=True)
        
        def implement_new_x (node, node_positions, barycenter_x, neighbors):
            node_positions[node] = (barycenter_x, node_positions[node][1])
            return node_positions
        
        def calculate_barycenter(node, parents, node_positions, neighbors):
            parents_x_sum = sum(node_positions[parent][0] for parent in parents)
            if parents_x_sum == 0:
                barycenter_x = node_positions[node][0]
            else:
                barycenter_x = parents_x_sum / len(parents)
#             print(node, neighbors, barycenter_x)
            node_positions = implement_new_x (node, node_positions, barycenter_x, neighbors)
            return node_positions
        
        #calculate first barycenter
        for layer in layers[:-1]:
            nodes_in_curr_layer = [node for node, (x, y) in node_positions.items() if y == layer]
            nodes_in_next_layer = [node for node, (x, y) in node_positions.items() if y == layer-1]
            has_parents = {node: [] for node in nodes_in_next_layer}
            for node in nodes_in_next_layer:
                for source, target, _ in edges:
                    if target == node:
                        has_parents[node].append(source)
#                 print(layer-1, "parents", has_parents)    
                node_positions = calculate_barycenter(node, has_parents[node], node_positions, nodes_in_next_layer)
                
            #recoordinate too close nodes
#             print(layer)
            node_groups = {}
            for node, pos in node_positions.items():
                x_coord, y_coord = pos
                if (x_coord, y_coord) in node_groups:
                    node_groups[(x_coord, y_coord)].append(node)
                else:
                    node_groups[(x_coord, y_coord)] = [node]
            too_close_groups = [group for group in node_groups.values() if len(group) > 1 and max(pos[0] for node in group) - min(pos[0] for node in group) < 1.2]
#             print(too_close_groups)
            # Divide each too close group with a spacing of 1
            for group in too_close_groups:
                group_size = len(too_close_groups)
                if group_size % 2 != 0: #if odd then middle node is middle
                    step_start = -(group_size-1)/2
                    for i, node in enumerate(group):
                        node_positions[node] = (node_positions[node][0] + step_start+i,  node_positions[node][1])
                else: #else 2 nodes are the middle
                    step_start = -(group_size)/2+0.5
                    for i, node in enumerate(group):
                        node_positions[node] = (node_positions[node][0] + step_start+i,  node_positions[node][1])
                
        return nodes, edges, node_positions
        
        
    cyclic_edges = guarantee_heuristic(nodes, edges, node_positions)
    #reverse the cyclic_edges using guarantee_heuristic
    edges = [(edge[1], edge[0], edge[2]) if edge in cyclic_edges else edge for edge in edges]
    
    print("Layer Assignment")
    #Improve arrow layers by assigning dummies
    nodes, edges, node_positions = layer_assignment(nodes, edges, root)
    draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)
    
    print("Crossing minimization")
    #Barycenter Heuristic:
    nodes, edges, node_positions = crossing_minimization(nodes, edges, node_positions, method='Barycenter')
    draw_node_link_diagram(nodes, edges, node_positions, node_colors, node_size)
    
    return node_positions

### Step 5

In [ ]:
def arrange_in_subgraphs(nodes, edges, graph):
    subgraphs = graph.split("subgraph")
    # in case of argumentatin network: subgraph "Youngest Devonian Strata" and subgraph "Gap in the Sequence of Devonshi"
    subgraph1 = subgraphs[6]
    subgraph2 = subgraphs[7]

    # per subgraph, get the corresponding nodes and edges
    sg1_nodes = []
    sg2_nodes = []
    for node in nodes:
        if subgraph1.__contains__(node):
            sg1_nodes.append(node)
        if subgraph2.__contains__(node):
            sg2_nodes.append(node)
    
    sg1_edges = []
    sg2_edges = []
    for edge in edges:
        if sg1_nodes.__contains__(edge[0]) and sg1_nodes.__contains__(edge[1]):
            sg1_edges.append(edge)
        if sg2_nodes.__contains__(edge[0]) and sg2_nodes.__contains__(edge[1]):
            sg2_edges.append(edge)
    
    node_positions1 = arrange_in_force(sg1_nodes,sg1_edges)
    node_positions2 = arrange_in_force(sg2_nodes,sg2_edges)
    
    # draw already, but better to move this outside later
    node_colors = {}
    for node1 in sg1_nodes:
        node_colors[node1] = "#" + ''.join(random.choices('0123456789ABCDEF', k=6)) 
    for node2 in sg2_nodes:
        node_colors[node2] = "#" + ''.join(random.choices('0123456789ABCDEF', k=6)) 

    inter_connecting_edges = [(source, target) for source, target in edges if ((source in sg1_nodes and target in sg2_nodes) or (source in sg2_nodes and target in sg1_nodes))]
    
    draw_two_node_link_diagram(sg1_edges, sg2_edges, node_positions1, node_positions2, node_colors, node_size=300, directed=False)
    return node_positions1, node_positions2

In [ ]:
one, two = arrange_in_subgraphs(nodes, edges, G)

## Step 6